In [58]:
# para la comparación y correción de texto

!pip3 install --quiet fuzzywuzzy python-Levenshtein

In [66]:
# para record linkage
!pip3 install --quiet recordlinkage

# Limpieza de datos
## Restricciones de tipo de datos


In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')

### String a entero o flotante


In [3]:
%%writefile /tmp/data.csv
orderId,price,percentage
1,100$,15.3%
2,120$,22.1%
3,128$,54.2%
4,155$,10.0%
5,234$,6%

Overwriting /tmp/data.csv


In [4]:
df = pd.read_csv("/tmp/data.csv")

#
# Note que los tipos de las columnas price y
# percentage son object. Se debe a los caracteres
# $ y % en el archivo.
#
#
display(
    df,
    df.dtypes
)

,orderId,price,percentage
0,1,100$,15.3%
1,2,120$,22.1%
2,3,128$,54.2%
3,4,155$,10.0%
4,5,234$,6%


orderId        int64
price         object
percentage    object
dtype: object

In [5]:
#
# Corrección
#
df.price = df.price.str.strip('$')
df.price = df.price.astype(int)

df.percentage = df.percentage.str.strip('%')
df.percentage = df.percentage.astype(float)

display(
    df,
    df.dtypes
)

,orderId,price,percentage
0,1,100,15.3
1,2,120,22.1
2,3,128,54.2
3,4,155,10.0
4,5,234,6.0


orderId         int64
price           int64
percentage    float64
dtype: object

### Numérico a categoría

Categorias:
```code
0  single
1  married
2  divorced
```

In [6]:
%%writefile /tmp/data.csv
personId,status
1,0
2,0
3,1
3,2
4,2

Overwriting /tmp/data.csv


In [7]:
df = pd.read_csv("/tmp/data.csv")

#
# El status es int64
#
display(
    df,
    df.dtypes
)

,personId,status
0,1,0
1,2,0
2,3,1
3,3,2
4,4,2


personId    int64
status      int64
dtype: object

In [8]:
df.status = df.status.astype('category')

display(
    df,
    df.dtypes
)

,personId,status
0,1,0
1,2,0
2,3,1
3,3,2
4,4,2


personId       int64
status      category
dtype: object

In [9]:
df.describe()

,personId
count,5.000000
mean,2.600000
std,1.140175
min,1.000000
25%,2.000000
50%,3.000000
75%,3.000000
max,4.000000


In [10]:
df.status

0    0
1    0
2    1
3    2
4    2
Name: status, dtype: category
Categories (3, int64): [0, 1, 2]

## Restricciones de rango de datos

### Rangos numéricos

In [11]:
%%writefile /tmp/data.csv
personId,rangecol
1,1
2,3
3,2
4,10
5,0
6,1
7,10
8,9

Overwriting /tmp/data.csv


Los valores de la columna ___rangecol___ están restringidos al rango [1, 2, 3].

In [12]:
df = pd.read_csv('/tmp/data.csv')

#
# registros que no cumplen la restricción.
#
df[(df.rangecol < 1) | (df.rangecol > 3)].rangecol

3    10
4     0
6    10
7     9
Name: rangecol, dtype: int64

In [13]:
#
# Se convierten valores > 3 a 3 y valoes < 1 a 1
#
df.rangecol[df.rangecol > 3] = 3
df.rangecol[df.rangecol < 1] = 1
df

,personId,rangecol
0,1,1
1,2,3
2,3,2
3,4,3
4,5,1
5,6,1
6,7,3
7,8,3


In [14]:
df = pd.read_csv('/tmp/data.csv')

#
# Borrado de registros que no están en el rango
# mediante selección
#
df = df[(df.rangecol >= 1) & (df.rangecol <= 3)]
df

,personId,rangecol
0,1,1
1,2,3
2,3,2
5,6,1


In [15]:
df = pd.read_csv("/tmp/data.csv")

#
# Borrado de registros que no están en el rango
# usando la función drop()
#
df.drop(
    df[(df.rangecol < 1) | (df.rangecol > 3)].index,
    inplace=True,
)
df

,personId,rangecol
0,1,1
1,2,3
2,3,2
5,6,1


### Rangos de fecha

In [16]:
%%writefile /tmp/data.csv
eventId,eventDate
1,2012-01-10
2,1900-12-23
3,2018-09-17
4,2019-11-15
5,2020-04-23
6,2025-07-03
7,2020-02-17
8,2017-08-12
9,2015-06-24

Overwriting /tmp/data.csv


In [17]:
df = pd.read_csv('/tmp/data.csv')

display(
    df,
    df.dtypes
)

,eventId,eventDate
0,1,2012-01-10
1,2,1900-12-23
2,3,2018-09-17
3,4,2019-11-15
4,5,2020-04-23
5,6,2025-07-03
6,7,2020-02-17
7,8,2017-08-12
8,9,2015-06-24


eventId       int64
eventDate    object
dtype: object

In [18]:
#
# Cambio del tipo de dato de 'eventDate' a datetime
#
df.eventDate = pd.to_datetime(df.eventDate)

display(
    df,
    df.dtypes
)

,eventId,eventDate
0,1,2012-01-10
1,2,1900-12-23
2,3,2018-09-17
3,4,2019-11-15
4,5,2020-04-23
5,6,2025-07-03
6,7,2020-02-17
7,8,2017-08-12
8,9,2015-06-24


eventId               int64
eventDate    datetime64[ns]
dtype: object

Rango de fechas:
```code
    1950-01-01
    Fecha actual
```

In [19]:
import datetime as dt

today = pd.to_datetime(dt.date.today())
today

Timestamp('2021-09-23 00:00:00')

In [20]:
#
# Verificación de restricciones
#
df[(df.eventDate < pd.to_datetime('1950-01-01')) | (df.eventDate > today)]

,eventId,eventDate
1,2,1900-12-23
5,6,2025-07-03


In [21]:
df.loc[
    df.eventDate < "1950-01-01",
    "eventDate",
] = pd.to_datetime("1950-01-01")


df.loc[
    df.eventDate > today,
    "eventDate"
] = today

display(
    df,
    df.dtypes
)

,eventId,eventDate
0,1,2012-01-10
1,2,1950-01-01
2,3,2018-09-17
3,4,2019-11-15
4,5,2020-04-23
5,6,2021-09-23
6,7,2020-02-17
7,8,2017-08-12
8,9,2015-06-24


eventId               int64
eventDate    datetime64[ns]
dtype: object

### Unicidad
__Posibilidades:__

- Registro completo duplicado.
- Algunos campos duplicados, demas campos con valores diferentes

__Soluciones posibles:__
- Borrado de los registros completamente duplicados.
- Agregación para campos con valores diferentes.

In [22]:
%%writefile /tmp/data.csv
clientId,name,phone,companyId
1,Hersch Szymanowicz,+237 (561) 702-4118,3
2,Gaven Brito,+51 (870) 799-1508,7
3,Maressa Pavlishchev,+62 (350) 377-8621,4
4,Corine Dunseith,+63 (797) 344-2571,8
5,Ramon Lawrence,+66 (557) 865-3845,6
6,Ibbie Whitehouse,+98 (495) 896-6408,1
7,Neils Capelen,+86 (361) 914-8734,6
8,Thia Malkie,+46 (564) 145-8997,4
9,Missy Folomkin,+33 (962) 798-0776,7
10,Eleanor Gallamore,+86 (366) 702-2334,5
7,Neils Capelen,+86 (361) 914-8734,6
4,Corine Dunseith,+63 (797) 344-2571,8
2,Gaven Brito,+51 (870) 799-2308,2
5,Ramon Lawrence,+66 (557) 061-3844,5
4,Corine Dunseith,+63 (797) 344-2571,8

Overwriting /tmp/data.csv


In [23]:
df = pd.read_csv('/tmp/data.csv')

display(
    df,
    df.dtypes
)

,clientId,name,phone,companyId
0,1,Hersch Szymanowicz,+237 (561) 702-4118,3
1,2,Gaven Brito,+51 (870) 799-1508,7
2,3,Maressa Pavlishchev,+62 (350) 377-8621,4
3,4,Corine Dunseith,+63 (797) 344-2571,8
4,5,Ramon Lawrence,+66 (557) 865-3845,6
5,6,Ibbie Whitehouse,+98 (495) 896-6408,1
6,7,Neils Capelen,+86 (361) 914-8734,6
7,8,Thia Malkie,+46 (564) 145-8997,4
8,9,Missy Folomkin,+33 (962) 798-0776,7
9,10,Eleanor Gallamore,+86 (366) 702-2334,5


clientId      int64
name         object
phone        object
companyId     int64
dtype: object

In [24]:
#
# La función duplicaated() indica si el registro
# completo está duplicado.
#
df.duplicated()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10     True
11     True
12    False
13    False
14     True
dtype: bool

In [25]:
#
# Visualización de los registros duplicados
#
df[df.duplicated(keep='first')]

,clientId,name,phone,companyId
10,7,Neils Capelen,+86 (361) 914-8734,6
11,4,Corine Dunseith,+63 (797) 344-2571,8
14,4,Corine Dunseith,+63 (797) 344-2571,8


In [26]:
#
# Visualización de los registros duplicados
#
df[df.duplicated(keep='last')]

,clientId,name,phone,companyId
3,4,Corine Dunseith,+63 (797) 344-2571,8
6,7,Neils Capelen,+86 (361) 914-8734,6
11,4,Corine Dunseith,+63 (797) 344-2571,8


In [27]:
#
# Visualización de los registros duplicados
#
df[
    df.duplicated(
        subset=["clientId", "name"],
        keep="last",
    )
]

,clientId,name,phone,companyId
1,2,Gaven Brito,+51 (870) 799-1508,7
3,4,Corine Dunseith,+63 (797) 344-2571,8
4,5,Ramon Lawrence,+66 (557) 865-3845,6
6,7,Neils Capelen,+86 (361) 914-8734,6
11,4,Corine Dunseith,+63 (797) 344-2571,8


In [28]:
#
# Visualización de los registros duplicados
#
df[
    df.duplicated(
        subset=["clientId", "name"],
        keep=False,
    )
].sort_values(by=['clientId', 'name'])

,clientId,name,phone,companyId
1,2,Gaven Brito,+51 (870) 799-1508,7
12,2,Gaven Brito,+51 (870) 799-2308,2
3,4,Corine Dunseith,+63 (797) 344-2571,8
11,4,Corine Dunseith,+63 (797) 344-2571,8
14,4,Corine Dunseith,+63 (797) 344-2571,8
4,5,Ramon Lawrence,+66 (557) 865-3845,6
13,5,Ramon Lawrence,+66 (557) 061-3844,5
6,7,Neils Capelen,+86 (361) 914-8734,6
10,7,Neils Capelen,+86 (361) 914-8734,6


In [29]:
#
# Borrado de registros duplicados
#
df[df.duplicated()]

,clientId,name,phone,companyId
10,7,Neils Capelen,+86 (361) 914-8734,6
11,4,Corine Dunseith,+63 (797) 344-2571,8
14,4,Corine Dunseith,+63 (797) 344-2571,8


In [30]:
df.drop_duplicates(inplace=True)
df

,clientId,name,phone,companyId
0,1,Hersch Szymanowicz,+237 (561) 702-4118,3
1,2,Gaven Brito,+51 (870) 799-1508,7
2,3,Maressa Pavlishchev,+62 (350) 377-8621,4
3,4,Corine Dunseith,+63 (797) 344-2571,8
4,5,Ramon Lawrence,+66 (557) 865-3845,6
5,6,Ibbie Whitehouse,+98 (495) 896-6408,1
6,7,Neils Capelen,+86 (361) 914-8734,6
7,8,Thia Malkie,+46 (564) 145-8997,4
8,9,Missy Folomkin,+33 (962) 798-0776,7
9,10,Eleanor Gallamore,+86 (366) 702-2334,5


## Restricciones de pertenencia

### Categorías inconsistentes

In [31]:
%%writefile /tmp/data.csv
personId,eventType
1,AA
2,A
3,AZ
4,AB
5,ZB
6,ZZ
7,BA
8,BB

Overwriting /tmp/data.csv


In [32]:
valid_eventType = {'AA', 'AB', 'BA', 'BB'}

df = pd.read_csv('/tmp/data.csv')

#
# Categorias inconsistentes
#
set(df.eventType).difference(valid_eventType)

{'A', 'AZ', 'ZB', 'ZZ'}

In [33]:
#
# Registros con categorias inconsistentes
#
df[~df.eventType.isin(valid_eventType)]

,personId,eventType
1,2,A
2,3,AZ
4,5,ZB
5,6,ZZ


Posibles soluciones:

- Borrado del registro.
- Reemplazo de las categorias inconsistentes
- Inferencia la categoria a partir de otros campos.

In [34]:
#
# Borrado de registros inconsistentes
#
df = df[df.eventType.isin(valid_eventType)]
df

,personId,eventType
0,1,AA
3,4,AB
6,7,BA
7,8,BB


### Consistencia de valores en variables categóricas

In [35]:
%%writefile /tmp/data.csv
personId,status
1,divorced_male
2,single_male
3,SINGLE_MALE
4,single_female
5,divorced_female
6,MARRIED_MALE
7,single female
8,single_male
9,divorced male

Overwriting /tmp/data.csv


In [36]:
df = pd.read_csv('/tmp/data.csv')

#
# Búsqueda de valores inconsistentes. Para
# columnas con muchos valores diferentes no
# funciona bien
#
df.status.value_counts()

single_male        2
single female      1
MARRIED_MALE       1
divorced_male      1
single_female      1
divorced_female    1
SINGLE_MALE        1
divorced male      1
Name: status, dtype: int64

In [37]:
status_df = pd.DataFrame({ 'status': list(df.status) })
status_df

,status
0,divorced_male
1,single_male
2,SINGLE_MALE
3,single_female
4,divorced_female
5,MARRIED_MALE
6,single female
7,single_male
8,divorced male


In [38]:
status_df = status_df.drop_duplicates()
status_df

,status
0,divorced_male
1,single_male
2,SINGLE_MALE
3,single_female
4,divorced_female
5,MARRIED_MALE
6,single female
8,divorced male


In [39]:
status_df['key'] = status_df.status.str.lower()
status_df

,status,key
0,divorced_male,divorced_male
1,single_male,single_male
2,SINGLE_MALE,single_male
3,single_female,single_female
4,divorced_female,divorced_female
5,MARRIED_MALE,married_male
6,single female,single female
8,divorced male,divorced male


In [40]:
status_df.groupby('key').agg(list)


,status
key,
divorced male,[divorced male]
divorced_female,[divorced_female]
divorced_male,[divorced_male]
married_male,[MARRIED_MALE]
single female,[single female]
single_female,[single_female]
single_male,"[single_male, SINGLE_MALE]"


In [41]:

# Corrección por reemplazo
#
df['status'] = df.status.str.replace(' ', '_')
df['status'] = df.status.str.lower()
df.status.value_counts()

single_male        3
divorced_male      2
single_female      2
married_male       1
divorced_female    1
Name: status, dtype: int64

### Reemplazo de valores numéricos por categorias

In [42]:
%%writefile /tmp/data.csv
salary
100
90
86
25
53
48
23
10
45
94
32
67
25
12
99

Overwriting /tmp/data.csv


In [43]:
#
# Rangos y categorias.
#
ranges = [0, 20, 40, 60, 80, np.inf]
names = ['0-19', '20-39', '40-59', '60-79', '80+']

df = pd.read_csv('/tmp/data.csv')

#
# Creación de la columna
#
df['group'] = pd.cut(
    df.salary,
    bins=ranges,
    labels=names,
)
df

,salary,group
0,100,80+
1,90,80+
2,86,80+
3,25,20-39
4,53,40-59
5,48,40-59
6,23,20-39
7,10,0-19
8,45,40-59
9,94,80+


### Reemplazo de strings por categorias

In [44]:
%%writefile /tmp/data.csv
vehicle
trailblazer
spark
bolt ev
trax
equinox
blazer
beat
joy
trailblazer
spark
bolt ev
traverse
bolt euv
tahoe
malibu
suburban
onix
trax
bolt ev
trax
equinox
blazer
equinox
blazer
beat
joy
traverse

Overwriting /tmp/data.csv


In [45]:
df = pd.read_csv("/tmp/data.csv")

#
# Categorias
#
mapping = {
    "beat": "Cars",
    "blazer": "SUVs",
    "bolt euv": "Electric",
    "bolt ev": "Electric",
    "equinox": "SUVs",
    "joy": "Cars",
    "malibu": "Cars",
    "onix": "Cars",
    "spark": "Cars",
    "suburban": "SUVs",
    "tahoe": "SUVs",
    "trailblazer": "SUVs",
    "traverse": "SUVs",
    "trax": "SUVs",
}

df['category'] = df.vehicle.replace(mapping)
df

,vehicle,category
0,trailblazer,SUVs
1,spark,Cars
2,bolt ev,Electric
3,trax,SUVs
4,equinox,SUVs
5,blazer,SUVs
6,beat,Cars
7,joy,Cars
8,trailblazer,SUVs
9,spark,Cars


## Limpieza de cadenas de texto

__Tipos de problemas:__

- Errores de digitación.
- Cumplimiento con un patrón (e-mails, telefonos, direcciones, …)
- Longitud mínima

El formato correcto del número telefónico es:

001##########

In [46]:
%%writefile /tmp/data.csv
phone
+001-693-708-9470
1-881-785-8561
+742-672-3143
+1.235.809.0341
+1 (808) 6256866
1 (993) 7090910
(858) 6725653
11 930 5833
+228 415 7806
892 144 6773
38728

Overwriting /tmp/data.csv


In [47]:
df = pd.read_csv('/tmp/data.csv')

#
# Resulta más apropiado visualizar la columna
# ordenada formando bloques
#
display(
    df,
    '',
    df.sort_values('phone'),
    '',
    df.sort_values('phone', key=lambda series: [len(x) for x in series]),
)

,phone
0,+001-693-708-9470
1,1-881-785-8561
2,+742-672-3143
3,+1.235.809.0341
4,+1 (808) 6256866
5,1 (993) 7090910
6,(858) 6725653
7,11 930 5833
8,+228 415 7806
9,892 144 6773


''

,phone
6,(858) 6725653
0,+001-693-708-9470
4,+1 (808) 6256866
3,+1.235.809.0341
8,+228 415 7806
2,+742-672-3143
5,1 (993) 7090910
1,1-881-785-8561
7,11 930 5833
10,38728


''

,phone
10,38728
7,11 930 5833
9,892 144 6773
2,+742-672-3143
6,(858) 6725653
8,+228 415 7806
1,1-881-785-8561
3,+1.235.809.0341
5,1 (993) 7090910
4,+1 (808) 6256866


In [48]:
#
# Eliminación del '+'
#
df.phone = df.phone.str.replace('+','')
df.sort_values('phone', key=lambda series: [len(x) for x in series])

,phone
10,38728
7,11 930 5833
2,742-672-3143
8,228 415 7806
9,892 144 6773
6,(858) 6725653
1,1-881-785-8561
3,1.235.809.0341
4,1 (808) 6256866
5,1 (993) 7090910


In [49]:
#
# Corrección de los números con "1..."
#
df.phone = df.phone.map(lambda x: "00" + x if re.search(r"^1[\s\.-]", x) else x)
df.sort_values('phone', key=lambda series: [len(x) for x in series])

,phone
10,38728
7,11 930 5833
2,742-672-3143
8,228 415 7806
9,892 144 6773
6,(858) 6725653
0,001-693-708-9470
1,001-881-785-8561
3,001.235.809.0341
4,001 (808) 6256866


In [50]:
#
# Reemplazo del '.'
#
df.phone = df.phone.str.replace('.','-')
df.sort_values('phone', key=lambda series: [len(x) for x in series])

,phone
10,38728
7,11 930 5833
2,742-672-3143
8,228 415 7806
9,892 144 6773
6,(858) 6725653
0,001-693-708-9470
1,001-881-785-8561
3,001-235-809-0341
4,001 (808) 6256866


In [51]:
#
# Reemplazo espacios en blanco
#
df.phone = df.phone.str.replace(' ','-')
df.sort_values('phone', key=lambda series: [len(x) for x in series])

,phone
10,38728
7,11-930-5833
2,742-672-3143
8,228-415-7806
9,892-144-6773
6,(858)-6725653
0,001-693-708-9470
1,001-881-785-8561
3,001-235-809-0341
4,001-(808)-6256866


In [52]:
#
# Remoción paréntesis
#
df.phone = df.phone.str.replace('[()]','')
df.sort_values('phone', key=lambda series: [len(x) for x in series])

,phone
10,38728
6,858-6725653
7,11-930-5833
2,742-672-3143
8,228-415-7806
9,892-144-6773
4,001-808-6256866
5,001-993-7090910
0,001-693-708-9470
1,001-881-785-8561


In [53]:
#
# Adición del 001 inicial
#
df.phone = df.phone.map(lambda x: "001-" + x if re.search(r"^\d{3}-\d{3}-\d{4}$", x) else x)
df.sort_values('phone', key=lambda series: [len(x) for x in series])

,phone
10,38728
6,858-6725653
7,11-930-5833
4,001-808-6256866
5,001-993-7090910
0,001-693-708-9470
1,001-881-785-8561
2,001-742-672-3143
3,001-235-809-0341
8,001-228-415-7806


In [54]:
#
# Segundo registro de la tabla anterior
#
df.phone = df.phone.map(lambda x: '001-' + x if re.search(r"^\d{3}-\d{7}$", x) else x)
df.sort_values('phone', key=lambda series: [len(x) for x in series])

,phone
10,38728
7,11-930-5833
4,001-808-6256866
5,001-993-7090910
6,001-858-6725653
0,001-693-708-9470
1,001-881-785-8561
2,001-742-672-3143
3,001-235-809-0341
8,001-228-415-7806


In [55]:
#
# Segundo registro de la tabla anterior
#
df.phone = df.phone.map(lambda x: '001-0' + x if re.search(r"^\d{2}-\d{3}-\d{4}$", x) else x)
df.sort_values('phone', key=lambda series: [len(x) for x in series])

,phone
10,38728
4,001-808-6256866
5,001-993-7090910
6,001-858-6725653
0,001-693-708-9470
1,001-881-785-8561
2,001-742-672-3143
3,001-235-809-0341
7,001-011-930-5833
8,001-228-415-7806


In [56]:
#
# Se corrigen los '-' donde es posible
#
df.phone = df.phone.map(lambda x: x.replace('-','') if re.search(r"^001-\d{3}-\d{3}-\d{4}$", x) else x)
df.phone = df.phone.map(lambda x: x.replace('-','') if re.search(r"^001-\d{3}-\d{7}$", x) else x)
df.sort_values('phone', key=lambda series: [len(x) for x in series])

,phone
10,38728
0,0016937089470
1,0018817858561
2,0017426723143
3,0012358090341
4,0018086256866
5,0019937090910
6,0018586725653
7,0010119305833
8,0012284157806


In [57]:
#
# Se reemplazan los números invalidos por Nan
#
df.phone = df.phone.map(lambda x: np.nan if len(x) != 13 else x)
df

,phone
0,0016937089470
1,0018817858561
2,0017426723143
3,0012358090341
4,0018086256866
5,0019937090910
6,0018586725653
7,0010119305833
8,0012284157806
9,0018921446773


## Comparación y corrección de texto

In [59]:
from fuzzywuzzy import fuzz, process

### Comparación de cadenas de texto

In [60]:
#
# Comparación de strings
#
display(
    fuzz.WRatio('hola mundo', 'hola mundo'),
    fuzz.WRatio('hola mundo', 'Hola mundo'),
    fuzz.WRatio('hola mundo', 'Hola Mundo'),
    fuzz.WRatio('hola mundo', 'hola mundo!')
)

100

100

100

100

In [61]:
#
# Comparación de strings
#
display(
    fuzz.WRatio('hola mundo!', 'hola mundo'),
    fuzz.WRatio('hola mundo', 'Hola mundo!'),
    fuzz.WRatio('hola mundo', 'Hola mundoo'),
    fuzz.WRatio('hola mundo', 'hola mundooo'),
    fuzz.WRatio('hola mundo', 'hola mundoooo'),
    fuzz.WRatio('hola mundo', 'hola'),
    fuzz.WRatio('hola', 'hola mundo')
)

100

100

95

91

87

90

90

In [62]:
#
# Comparación con arrays
#
base_string = "hola mundo"
array = pd.Series(
    [
        "Hola mundo",
        "mundo",
        "Hola",
        "Hola mundo cruel!",
    ]
)


process.extract(
    base_string,
    array,
)

[('Hola mundo', 100, 0),
 ('mundo', 90, 1),
 ('Hola', 90, 2),
 ('Hola mundo cruel!', 90, 3)]

### Corrección y unificación básica cuando se conoce la clave

In [63]:
%%writefile /tmp/data.csv
ciudad
Medellin
Bogota
bogota
bogote
Mdellin
medellin
Cali
cali
CALI

Overwriting /tmp/data.csv


In [64]:
df = pd.read_csv('/tmp/data.csv')
df

,ciudad
0,Medellin
1,Bogota
2,bogota
3,bogote
4,Mdellin
5,medellin
6,Cali
7,cali
8,CALI


In [65]:
#
# Nombres validos
#
valid_names = ["Medellin", "Bogota", "Cali"]
df["ciudad_"] = df.ciudad.copy()

#
# Valor mínimo de similitud para hacer el cambio
#
min_threshold = 80

#
# Estructuta básica
#
for valid_name in valid_names:

    potential_matches = process.extract(
        valid_name,
        df.ciudad,
        limit=df.shape[0],
    )

    for potential_match in potential_matches:

        if potential_match[1] >= min_threshold:

            df.loc[df.ciudad == potential_match[0], "ciudad_"] = valid_name

df

,ciudad,ciudad_
0,Medellin,Medellin
1,Bogota,Bogota
2,bogota,Bogota
3,bogote,Bogota
4,Mdellin,Medellin
5,medellin,Medellin
6,Cali,Cali
7,cali,Cali
8,CALI,Cali


## Vinculación de registros (record linkage)

Se usa cuando no hay campos clave por los cuales se pueda hacer un join.

In [67]:
%%writefile /tmp/dfA.csv
first_name,last_name,birtdate,phone
Kayne M,Taffie,2/10/1985,+86 (669) 916-2473
Daisey S,Heisham,5/19/1990,+55 (858) 758-7630
Clair W,Brik,10/3/1976,+351 (509) 289-3191
Kippy L,Frome,9/18/1992,+420 (195) 491-9791
Burgess Jr,Klimes,2/4/1977,+86 (762) 990-4484
Dermot R,Garwill,9/27/1984,+86 (699) 948-9318
Hadley P,Gosker,2/15/1993,+48 (457) 883-3998
S Jackqueline,Papes,6/18/1983,+86 (784) 978-0726

Overwriting /tmp/dfA.csv


El segundo conjunto de datos contiene registros duplicados del primero.

In [68]:
%%writefile /tmp/dfB.csv
first_name,last_name,birtdate,phone
BURGESS,Klimes,2/4/1977,+86 (762) 990-4484
Dermot,Garwill,9/27/1984,+86 (699) 948-9318
Hadley,GOSKER,2/15/1993,+48 (457) 883-3998
Jackqueline,Papes,6/18/1983,+86 (784) 978-0726
Alastair,Barge,3/9/1971,+33 (182) 729-8581
Theobald,Bastian,11/15/1987,+62 (397) 242-4366
Pammi,Daffey,9/5/1986,+86 (761) 567-4803
Marcus,Charlo,7/7/1974,+86 (928) 602-4540
Burgess,KLIMES,2/4/1977,+86 (762) 990-4484
Dermot,Garwill,9/27/1984,+86 (699) 948-9318

Overwriting /tmp/dfB.csv


In [69]:
dfA = pd.read_csv('/tmp/dfA.csv')
dfB = pd.read_csv('/tmp/dfB.csv')

display(
    dfA,
    dfB
)

,first_name,last_name,birtdate,phone
0,Kayne M,Taffie,2/10/1985,+86 (669) 916-2473
1,Daisey S,Heisham,5/19/1990,+55 (858) 758-7630
2,Clair W,Brik,10/3/1976,+351 (509) 289-3191
3,Kippy L,Frome,9/18/1992,+420 (195) 491-9791
4,Burgess Jr,Klimes,2/4/1977,+86 (762) 990-4484
5,Dermot R,Garwill,9/27/1984,+86 (699) 948-9318
6,Hadley P,Gosker,2/15/1993,+48 (457) 883-3998
7,S Jackqueline,Papes,6/18/1983,+86 (784) 978-0726


,first_name,last_name,birtdate,phone
0,BURGESS,Klimes,2/4/1977,+86 (762) 990-4484
1,Dermot,Garwill,9/27/1984,+86 (699) 948-9318
2,Hadley,GOSKER,2/15/1993,+48 (457) 883-3998
3,Jackqueline,Papes,6/18/1983,+86 (784) 978-0726
4,Alastair,Barge,3/9/1971,+33 (182) 729-8581
5,Theobald,Bastian,11/15/1987,+62 (397) 242-4366
6,Pammi,Daffey,9/5/1986,+86 (761) 567-4803
7,Marcus,Charlo,7/7/1974,+86 (928) 602-4540
8,Burgess,KLIMES,2/4/1977,+86 (762) 990-4484
9,Dermot,Garwill,9/27/1984,+86 (699) 948-9318


### Data Cleaning

In [70]:
from recordlinkage.preprocessing import clean, phonenumbers

dfA.first_name = clean(dfA.first_name)
dfB.first_name = clean(dfB.first_name)

dfA.last_name = clean(dfA.last_name)
dfB.last_name = clean(dfB.last_name)

dfA.phone = phonenumbers(dfA.phone)
dfB.phone = phonenumbers(dfB.phone)

display(
    dfA,
    dfB
)

,first_name,last_name,birtdate,phone
0,kayne m,taffie,2/10/1985,+866699162473
1,daisey s,heisham,5/19/1990,+558587587630
2,clair w,brik,10/3/1976,+3515092893191
3,kippy l,frome,9/18/1992,+4201954919791
4,burgess jr,klimes,2/4/1977,+867629904484
5,dermot r,garwill,9/27/1984,+866999489318
6,hadley p,gosker,2/15/1993,+484578833998
7,s jackqueline,papes,6/18/1983,+867849780726


,first_name,last_name,birtdate,phone
0,burgess,klimes,2/4/1977,+867629904484
1,dermot,garwill,9/27/1984,+866999489318
2,hadley,gosker,2/15/1993,+484578833998
3,jackqueline,papes,6/18/1983,+867849780726
4,alastair,barge,3/9/1971,+331827298581
5,theobald,bastian,11/15/1987,+623972424366
6,pammi,daffey,9/5/1986,+867615674803
7,marcus,charlo,7/7/1974,+869286024540
8,burgess,klimes,2/4/1977,+867629904484
9,dermot,garwill,9/27/1984,+866999489318


In [71]:
#
# Creacion de pares de registros usando full()
# ===============================================
# Crea pares dfA x dfB y retorna el indice
#
import recordlinkage

indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(dfA, dfB)
pairs[:5], pairs[-5:]

(MultiIndex([(0, 0),
             (0, 1),
             (0, 2),
             (0, 3),
             (0, 4)],
            ),
 MultiIndex([(7, 5),
             (7, 6),
             (7, 7),
             (7, 8),
             (7, 9)],
            ))

In [72]:
#
# Formación de pares por atributos iguales.
#
indexer = recordlinkage.Index()
indexer.block('last_name')
candidate_links = indexer.index(dfA, dfB)
candidate_links

MultiIndex([(4, 0),
            (4, 8),
            (5, 1),
            (5, 9),
            (6, 2),
            (7, 3)],
           )

In [73]:
#
# Comparación de registros
#
compare_cl = recordlinkage.Compare()

compare_cl.exact(
    "last_name",
    "last_name",
    label="last_name",
)

compare_cl.string(
    "first_name",
    "first_name",
    method="jarowinkler",
    threshold=0.85,
    label="first_name",
)

features = compare_cl.compute(candidate_links, dfA, dfB)
features

last_name  first_name
4 0          1         1.0
  8          1         1.0
5 1          1         1.0
  9          1         1.0
6 2          1         1.0
7 3          1         1.0

In [74]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0    6
dtype: int64

In [75]:
#
# Se puede establecer un minimo para considerar
# que un grupo de registros son el mismo.
#
potential_matches = features[features.sum(axis=1) >= 1]
potential_matches

last_name  first_name
4 0          1         1.0
  8          1         1.0
5 1          1         1.0
  9          1         1.0
6 2          1         1.0
7 3          1         1.0

In [76]:
#
# Indices obtenidos
potential_matches.index

MultiIndex([(4, 0),
            (4, 8),
            (5, 1),
            (5, 9),
            (6, 2),
            (7, 3)],
           )

In [77]:
#
# Indices de los registros duplicados en el
# segundo dataframe
#
duplicate_rows = potential_matches.index.get_level_values(1)
duplicate_rows

Int64Index([0, 8, 1, 9, 2, 3], dtype='int64')

In [78]:
#
# Registros duplicados en dfB
#
dfB[dfB.index.isin(duplicate_rows)]

,first_name,last_name,birtdate,phone
0,burgess,klimes,2/4/1977,+867629904484
1,dermot,garwill,9/27/1984,+866999489318
2,hadley,gosker,2/15/1993,+484578833998
3,jackqueline,papes,6/18/1983,+867849780726
8,burgess,klimes,2/4/1977,+867629904484
9,dermot,garwill,9/27/1984,+866999489318


In [79]:
#
# Registros no duplicados en dfB
#
dfB_new = dfB[~dfB.index.isin(duplicate_rows)]
dfB_new

,first_name,last_name,birtdate,phone
4,alastair,barge,3/9/1971,+331827298581
5,theobald,bastian,11/15/1987,+623972424366
6,pammi,daffey,9/5/1986,+867615674803
7,marcus,charlo,7/7/1974,+869286024540


In [80]:
#
# Concatenación de dfA y dfB
#
dfA_new = dfA.append(dfB_new, ignore_index=True, sort=True)
dfA_new.sort_values('birtdate')

,birtdate,first_name,last_name,phone
2,10/3/1976,clair w,brik,+3515092893191
9,11/15/1987,theobald,bastian,+623972424366
0,2/10/1985,kayne m,taffie,+866699162473
6,2/15/1993,hadley p,gosker,+484578833998
4,2/4/1977,burgess jr,klimes,+867629904484
8,3/9/1971,alastair,barge,+331827298581
1,5/19/1990,daisey s,heisham,+558587587630
7,6/18/1983,s jackqueline,papes,+867849780726
11,7/7/1974,marcus,charlo,+869286024540
3,9/18/1992,kippy l,frome,+4201954919791
